In [ ]:
#enter your csv filename here
import pandas as pd
df = pd.read_csv('Fintro_Twitter_2016.csv', delimiter='\t')


In [ ]:
#edit the prompt here
completion_text_conversation = """
Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT: Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Cover_Edition\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"JURY: Age\", \"CONTENDER: Voice_Language-Use\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"CONTENDER: Age\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: MUSIC_Music\", \"ALLO-REFERENCES: General\", \"ALLO-REFERENCES: OTHER_Person\". Any reference to one of the names Kathleen Cools, Bart Vanegeren, Edith Aerts, Jeroen van Kan, Danny Theuwis, Guinevere Claeys, Adil El Arbi needs to be classified as one of the \"JURY\" aspect categories. Any reference to either one of the following names needs to be classified as one of the \"CONTENDER\" aspect categories: Jan Brokken, Saskia De Coster, Chris De Stoop, Stephan Enter, Arjen Fortuin, Philip Huff, Willem Otterspeer, Jamal Ouariachi, Connie Palmen, Hagar Peeters, Yves Petry, Inge Schilperoord, K. Schippers, Pieter Steinz, P.F. Thomése, Joke van Leeuwen, Annelies Verbeke, Peter Verhelst, L.H. Wiener, Maartje Wortel. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
"""

In [ ]:
import os
import openai
import subprocess
import requests
from typing import List, Optional
from pprint import pprint
os.environ["OPENAI_API_KEY"] = "yourkey" #be aware that this will cost, that tokens in some languages are more expensive etc.

API_KEY = os.getenv("OPENAI_API_KEY")

def print_response(response):
    if response.status_code == 200:
        print(response.json())
        return response.json()
    else:
        error = response.json()
        print(f"Error: {error['error']}")
        return error


In [ ]:
import os 
import logging
import sys

# Setup OPENAI_API_KEY

# Setup logging

log = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)s | %(levelname)s | %(message)s", level=logging.INFO)

# Update sys.path (or use PYTHONPATH)

#sys.path.insert(0, '..')

In [ ]:
from random import choice
from tqdm.notebook import tqdm
from json import loads
from pprint import pprint
from textwrap import dedent

analysis_results = []
extra_prompts = []

logging.getLogger("openai").setLevel(logging.INFO)

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}",
}

response = requests.post("https://api.openai.com/v1/completions", \
    headers=headers, json={
        "model": "gpt-3.5-turbo",
        "prompt": "Say this is a test",
        "temperature": 0.2,
        "max_tokens": 1000
    }
)

def create_completion(messages:str, suffix:str = None, model: str = "gpt-3.5-turbo", max_tokens: Optional[int] = 500, temperature: float = 1, top_p: Optional[int] = 1, n: Optional[int] = 1,  stream: Optional[bool] = False, echo: Optional[bool] = False):
    """
    https://platform.openai.com/docs/api-reference/completions/create
    ##### Given a prompt, the model will return one or more predicted completions, and can also return the probabilities of alternative tokens at each position.

    Args:
        - prompt (str): 
            - The prompt(s) to generate completions for, encoded as a string, array of strings, array of tokens, or array of token arrays. 
            Note that <|endoftext|> is the document separator that the model sees during training, so if a prompt is not specified the model will generate as if from the beginning of a new document. 
        
        - suffix (str): 
            - The suffix that comes after a completion of inserted text.
        
        - model (str, required): 
            - ID of the model to use. You can use the **List models** API to see all of your available models, or see our **Model overview** for descriptions of them. Defaults to "text-davinci-003".
        
        - max_tokens (int, optional, Default: 500): 
            - The maximum number of tokens to generate in the completion.
                The token count of your prompt plus `max_tokens` cannot exceed the model's context length. Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Defaults to 500.
        
        - temperature (float, optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            We generally recommend altering this or `top_p` but not both. Defaults to 0.
            
        - top_p (float, optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.
            
        - n (int, optional, Default: 1):
            - The number of completions to generate. Defaults to 1. 
        
        - stream (bool, optional): 
            - Whether to stream back partial progress. If set, tokens will be sent as data-only server-sent events as they become available, with the stream terminated by a `data: [DONE]` message. Defaults to False.
        
        - echo (bool, optional): _description_. Defaults to False.
        

    Returns:
        _type_: _description_
    """    
    response = requests.post("https://api.openai.com/v1/completions", \
        headers=headers, json={
            "model": model,
            "messages": messages,
            "suffix": suffix,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stream": stream,
            "echo": echo,
            "stop": suffix,
            "n": n,
            "top_p": top_p
        }
    )
    return response
    
def edit_prompt(instruction: str, input: Optional[str] = None, model: str = "text-davinci-edit-001", num: Optional[int] = 1, temperature: Optional[float] = 1, top_p: Optional[float] = 1):
    """#### Given a prompt and an instruction, the model will return an edited version of the prompt.
    https://platform.openai.com/docs/api-reference/edits
    Args:
        - instruction (str, required): 
            - The instruction that tells the model how to edit the prompt.
            
        - input (Optional[str], optional): 
            - The input text to use as a starting point for the edit. Defaults to None.
        
        - model (str, required): 
            - ID of the model to use. You can use the `text-davinci-edit-001` or `code-davinci-edit-001` model with this endpoint. Defaults to "text-davinci-edit-001".

        - n (Optional[int], optional): 
            - How many edits to generate for the input and instruction. Defaults to 1.

        - temperature (Optional[float], optional): 
            - What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.
            - We generally recommend altering this or `top_p` but not both. Defaults to 1.
        
        - top_p (Optional[float], optional):
            - An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
            - We generally recommend altering this or `temperature` but not both. Defaults to 1.

    Returns:
        - response
    """    
    response = requests.post("https://api.openai.com/v1/edits", \
        headers=headers, json={
            "model": model,
            "input": input,
            "instruction": instruction,
            "n": num,
            "temperature": temperature,
            "top_p": top_p
        }
    )
    return response

In [ ]:
df.head()

In [ ]:
df.rename(columns={'Caption': 'text', 'Filename': 'title'}, inplace=True)

In [ ]:
df.head()

In [ ]:
import time 
import traceback
from textwrap import dedent
from random import choice
from tqdm.notebook import tqdm
#from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent
import time
from json import JSONDecodeError, loads
from tqdm import tqdm

retry_attempts = 3
retry_delay = 2
analysis_results = []
extra_prompts = []


logging.getLogger("openai").setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)
for i, row in tqdm(df.iterrows(), total=len(df), desc="Analyzing reviews"):
    try:
        if pd.notnull(row["title"]):
            title = row["title"]
            text = row["text"]
            log.info(f"Analyzing feedback - \nID: {row['title']}\nTitle: {title}\nText: {text}\n")

            res = create_completion(
                model="gpt-3.5-turbo",
                temperature="0.2",
                messages=[
                    {"role": "system",    "content": "You are a data analyst that provides multi-target quadruple sentiment extraction."},
                    {"role": "system",    "content": "You annotate multiple opinion targets per line if the text contains more than one target. You annotate every single target only once. If the input text does not seem to contain any opinion targets or expressions, assume as opinion target the first noun and as expression the rest of the utterance. Hashtags and weblinks are not targets. Do not invent aspect categories or aspect terms yourself."},
                    {"role": "user",      "content": completion_text_conversation + text + ":"},    
                    #{"role": "user",      "content": "What is the most likely "},
                    #{"role": "user",      "content": "Where are most of the students from?"},
                    #{"role": "user",      "content": "Which extra-curricular activity/activities is most popular? Which among them has the highest female participation?"},
                    # add in as many questions as you want
                ],
            )

            try:
                raw_json = res.json()["choices"][0]["message"]["content"].strip()
                #print(raw_json)
                json_data = loads(raw_json)
                analysis_results.append(json_data)

                log.debug(f"JSON response: {pprint(json_data)}")

                extra_prompts.append(f"\n{text}\n{raw_json}")

            except JSONDecodeError as e:
                if "Unfortunately, it seems that there is no text provided in your request for me to perform sentiment extraction on." in str(e):
                    log.error("No text provided for sentiment analysis")
                    analysis_results.append("No text provided for sentiment analysis")
                elif "Expecting value" in str(e):
                    if "Unterminated string" in str(e):
                        log.warning("JSON string is unterminated")
                        analysis_results.append(raw_json)
                    elif "I'm sorry, but I need some text to perform the sentiment extraction" in str(e):
                        log.error("No text provided for sentiment analysis")
                        analysis_results.append("No text provided for sentiment analysis")
                    else:
                        log.error(f"Failed to parse '{raw_json}' -> {e}")
                        analysis_results.append([])
                else:
                    log.error(f"Failed to parse '{raw_json}' -> {e}")
                    analysis_results.append([])
            except Exception as e:
                log.error(f"Error performing sentiment analysis on text: {text}")
                log.error(str(e))
                analysis_results.append([])
        else:
            log.warning(f"No id found in row {i}. Skipping analysis.")
            analysis_results.append([])
    except Exception as e:
        traceback.print_exc()
        log.warning(f"The error ecountered, {i}. Skipping analysis.")
        analysis_results.append([])
        
    #time.sleep(2) #added 2 seconds delay between requests

df["analysis"] = analysis_results
#df.to_csv("./feedbacks_analysis_2015_jury_batch1.csv", index=False)




In [ ]:
print(df.columns)

In [ ]:
#make sure to rename this, or your results file will get overwritten
df.to_csv("./feedbacks_analysis_2016_fintro.csv", index=False)


In [ ]:
#Prompt: Please perform quadruple sentiment extraction on this utterance by extracting \"Aspect Category\", \"Aspect Term\", \"Opinion Target\", \"Opinion Expression\", \"Sentiment\" and \"Confidence Score\" using the original language of the text. The aspect categories and their terms are: \"TEXT: Title\", \"TEXT: Quote\", \"TEXT: PoV_Narration\", \"TEXT: Motifs_Themes\", \"TEXT: Language_Style_\", \"TEXT: General-Content_Plot\", \"TEXT: General\", \"TEXT: Form\", \"TEXT: Flow_Rhythm_Punctuation\", \"TEXT: Cover_Edition\", \"TEXT: Characters\", \"READING: Pronunciation_Intonation_Understandability\", \"READING: General\", \"READING: Flow_Rhythm_Punctuation\", \"ONSITE-AUDIENCE: General\", \"ONSITE-AUDIENCE: Behaviour\", \"ONSITE-AUDIENCE: Appearance_Clothing\", \"ONSITE-AUDIENCE: Age\", \"META: Winner_Award-Ceremony\", \"META: Weather\", \"META: Voting\", \"META: Videoportrait\", \"META: Technology_Social-Media\", \"META: Side-Event\", \"META: Shortlist\", \"META: Opening-Speech\", \"META: Online-Assessment\", \"META: Music\", \"META: Montage\", \"META: Main-Event\", \"META: Longlist\", \"META: Location\", \"META: Literature_Literary-Prizes\", \"JURY: Voice_Language-Use\", \"JURY: Quote\", \"JURY: General\", \"JURY: Discussion_Valuation\", \"JURY: Behaviour\", \"JURY: Appearance_Clothing\", \"JURY: Age\", \"CONTENDER: Voice_Language-Use\", \"CONTENDER: Quote\", \"CONTENDER: General\", \"CONTENDER: Gender\", \"CONTENDER: Appearance_Clothing\", \"CONTENDER: Age\", \"ALLO-REFERENCES: TEXT_Other-Text\", \"ALLO-REFERENCES: TEXT_Other-Author\", \"ALLO-REFERENCES: SCREEN_Film_Tv\", \"ALLO-REFERENCES: SCREEN_Director_Actor\", \"ALLO-REFERENCES: MUSIC_Musician\", \"ALLO-REFERENCES: MUSIC_Music\", \"ALLO-REFERENCES: General\", \"ALLO-REFERENCES: OTHER_Person\". Any reference to one of the names Kathleen Cools, Bart Vanegeren, Edith Aerts, Jeroen van Kan, Danny Theuwis, Guinevere Claeys, Adil El Arbi needs to be classified as one of the \"JURY\" aspect categories. Any reference to either one of the following names needs to be classified as one of the \"CONTENDER\" aspect categories: Jan Brokken, Saskia De Coster, Chris De Stoop, Stephan Enter, Arjen Fortuin, Philip Huff, Willem Otterspeer, Jamal Ouariachi, Connie Palmen, Hagar Peeters, Yves Petry, Inge Schilperoord, K. Schippers, Pieter Steinz, P.F. Thomése, Joke van Leeuwen, Annelies Verbeke, Peter Verhelst, L.H. Wiener, Maartje Wortel. Any other name is to be seen as belonging to the Aspect Category \"ALLO-REFERENCE\". The Sentiment is  \"positive\", \"neutral\" or \"negative\". The numeric confidence score ranges between -1 and 1 and is rounded to hundreds and tens. The output is a single JSON object that contains a list of JSON objects. The confidence score is represented as a string in the output. The order of the list of JSON objects is the same as the order in which the opinions were expressed in the text. Ignore the hashtags and the weblinks. Reply only with the json. Do not explain. Analyze sentiment for each opinion target separately. All entities mentioned in the text do belong to at least one of the aspect categories and terms.\n\n
